Scrapes the novelupdates webpage, http://www.novelupdates.com/
for novel information and cleans it.
Finally saves the dataset as a csv file.

Didn't find any easy way to get all the novel ids, hence, the novels listing is first used to get all ids.

In [33]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

novel_list_page = "http://www.novelupdates.com/novelslisting/?st=1&pg="
novel_page = "http://www.novelupdates.com/?p="

In [34]:
# Get the maximum number of novel pages from the listings

def get_novel_list_max_pages(page):
    soup = BeautifulSoup(page.content, 'html.parser')
    dig_pag = soup.find('div', attrs={'class':'digg_pagination'})
    page_links = dig_pag.find_all('a')
    last_page_link = str(page_links[2]) # The last page is the 3rd
    num = re.search('pg=\d+', last_page_link).group()[3:]
    return int(num)

page = requests.get(novel_list_page + '1')
novels_max_pages = get_novel_list_max_pages(page)
print("Pages with novels: " + str(novels_max_pages))

Pages with novels: 123


In [35]:
# 1 For testing
novels_max_pages = 1

# Get all novel ids from the novels listing
def get_novel_ids(page):
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find('table', attrs={'id':'myTable'})
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    
    novel_ids = []
    for row in rows:
        col = row.find_all('td')[-1]
        novel_id = col.a['id'][3:]
        novel_ids.append(novel_id)
    return novel_ids

all_novel_ids = []
for i in range(1,novels_max_pages+1):
    page = requests.get(novel_list_page + str(i))
    novel_ids = get_novel_ids(page)
    all_novel_ids.extend(novel_ids)
    time.sleep(1)

df = pd.DataFrame(all_novel_ids, columns=['id'])

In [37]:
def parse_novel_page(id):
    page = requests.get(novel_page + str(id))
#    page = requests.get(id) # TEMP
    soup = BeautifulSoup(page.content, 'html.parser')
    wrapper = soup.find('div', attrs={'class': 'wpb_wrapper'})
    
    data = {}
    #tags =
    
    genres = [genre.text
              for genre in wrapper
                .find('div', attrs={'id': 'seriesgenre'})
                .find_all('a', attrs={'class': 'genre'})]
    
    rating_text = wrapper.find('span', attrs={'class': 'uvotes'}).text.split(' ')
    rating = float(rating_text[0][1:])
    rating_votes = int(rating_text[3])
    
    data['genres'] = genres
    data['rating'] = rating
    data['rating_votes'] = rating_votes
    
    #langauge =
    #author = 
    #start_year = 
    #volumes_total =
    #chapters_total =
    #chapters_translated =
    #complete
    #complete_translated =
    #licensed = 
    #original_publisher =
    #english_publisher =
    #release_freq = 
    #activity_week_rank =
    #activity_month_rank =
    #activity_all_time_rank =
    #on_reading_lists =
    #reading_list_month_rank =
    #reading_list_all_time_rank = 
    #assoc_names = 
    #related_series_ids =
    #recommended_series_ids =
    
    
    # return these as a map (name -> result)
    # add these as new columns to the df
    time.sleep(1)
    return pd.Series(data)

#https://stackoverflow.com/questions/16236684/apply-pandas-function-to-column-to-create-multiple-new-columns
#parse_novel_page('http://www.novelupdates.com/series/i-shall-seal-the-heavens/')

#df.merge(df.id.apply(lambda x: parse_novel_page(x)), left_index=True, right_index=True)
pd.merge(df, df.id.apply(lambda x: parse_novel_page(x)), left_index=True, right_index=True)

,id,genres,rating,rating_votes
0,1173,"[Adventure, Fantasy, Sci-fi]",2.9,25
1,1175,"[Action, Sci-fi, Shounen]",2.8,12
2,1177,"[Action, Adventure, Fantasy, Sci-fi, Shounen]",2.7,15
3,13038,"[Action, Adventure, Fantasy, Mystery, Psycholo...",3.0,3
4,1171,"[Drama, Mystery, Psychological, Romance, Schoo...",3.7,30
5,2042,"[Action, Comedy, Drama, School Life, Shounen, ...",2.4,10
6,6612,"[Action, Adventure, Comedy, Drama, Fantasy, Sc...",3.6,550
7,11133,"[Comedy, Romance, School Life, Shounen Ai, Sli...",4.6,147
8,9343,"[Adult, Adventure, Comedy, Fantasy, Gender Ben...",4.0,141
9,8233,"[Action, Comedy, Fantasy, Sci-fi]",4.5,133
